<a href="https://colab.research.google.com/github/bocekout/March-2025-hackathon/blob/main/Mar25_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install uv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 54.7 MB/s eta 0:00:00


In [ ]:
pip install llama-index==0.11.6 llama-index-llms-openai llama-index-readers-file llama-index-embeddings-openai llama-index-llms-openai-like "openinference-instrumentation-llama-index>=2" arize-phoenix python-dotenv

In [ ]:
from os import environ
from dotenv import load_dotenv

KeyError: 'OPENAI_API_KEY'

In [ ]:
load_dotenv()

OPENAI_API_KEY = environ["OPENAI_API_KEY"]

In [ ]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI


# Create an llm object to use for the QueryEngine and the ReActAgent
llm = OpenAI(model="gpt-4")

In [ ]:
import phoenix as px
session = px.launch_app()

🌍 To view the Phoenix app in your browser, visit https://nwc1t9erp71-496ff2e9c6d22116-6006-colab.googleusercontent.com/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [ ]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from phoenix.otel import register

tracer_provider = register()
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [ ]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [ ]:
if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs, show_progress=True)
    uber_index = VectorStoreIndex.from_documents(uber_docs, swow_progress=True)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

Parsing nodes:   0%|          | 0/238 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/343 [00:00<?, ?it/s]

In [ ]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3, llm=llm)
uber_engine = uber_index.as_query_engine(similarity_top_k=3, llm=llm)

In [ ]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

In [ ]:
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    max_turns=10,
)

In [ ]:
response = agent.chat("Who had more profit in 2021, Lyft or Uber?")
print(str(response))

> Running step 4871e622-4030-47ec-86a3-13cb20e607f5. Step input: Who had more profit in 2021, Lyft or Uber?
Thought: The current language of the user is English. I need to use the tools to help me answer the question. I will first use the lyft_10k tool to get the profit of Lyft in 2021, then I will use the uber_10k tool to get the profit of Uber in 2021.
Action: lyft_10k
Action Input: {'input': "What was Lyft's profit in 2021?"}
Observation: Lyft did not make a profit in 2021. The company reported a net loss of $1,009,359,000 for the year.
> Running step 9ef5ca8d-8ede-4693-98b1-e08a8743cef5. Step input: None
Thought: Lyft reported a net loss in 2021. Now I need to find out Uber's profit for the same year to compare.
Action: uber_10k
Action Input: {'input': "What was Uber's profit in 2021?"}
Observation: Uber did not make a profit in 2021. Instead, it reported a net loss of $496 million.
> Running step 329c0954-0d3f-4ad5-bf6a-e8b3b1d2ee92. Step input: None
Thought: I can answer without 